In [4]:
import sys
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork
import torch as T
import torch.nn as nn 

In [5]:
N= CriticNetwork(5, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='tb', learning_rate=0.02)

bt= T.tensor([[-1.4355, -0.7806,  0.3042,  1.1601, -0.1184]])
at= T.tensor([[0.8233, 0.8126]])
for target_param in N.parameters():
    print(target_param)
bt, at, N(bt, at)

Parameter containing:
tensor([-0.2044, -0.0149,  0.1442,  0.1900,  0.2032, -0.3229, -0.3135, -0.1537,
        -0.1600,  0.2402], requires_grad=True)
Parameter containing:
tensor([[0.4629],
        [0.7502],
        [0.7552],
        [0.5001],
        [0.5373],
        [0.7027],
        [0.6646],
        [0.6163],
        [0.4878],
        [0.4298]], requires_grad=True)
Parameter containing:
tensor([[ 0.0175, -0.3737,  0.0772,  0.1259,  0.1587,  0.0358, -0.1614],
        [ 0.2866, -0.0279, -0.2935, -0.3641, -0.3645, -0.3554, -0.0442],
        [-0.3558,  0.2212, -0.3520, -0.1743,  0.2744,  0.3363,  0.2285],
        [-0.1083,  0.0695,  0.1284, -0.3024, -0.1171, -0.2703,  0.1969],
        [-0.2650, -0.3011,  0.2865,  0.1268, -0.0164, -0.0051,  0.1715],
        [ 0.3289,  0.1176, -0.3321, -0.3122, -0.0036, -0.1950,  0.3550],
        [-0.3253, -0.0130, -0.0779, -0.0731,  0.3058,  0.3601, -0.3181],
        [-0.2788,  0.3207, -0.1983,  0.1868, -0.0632,  0.2583, -0.2331],
        [ 0.2896,  0.3

(tensor([[-1.4355, -0.7806,  0.3042,  1.1601, -0.1184]]),
 tensor([[0.8233, 0.8126]]),
 tensor([[-0.1284]], grad_fn=<AddmmBackward0>))

In [6]:
Nt= ActorNetwork(5, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='btt', learning_rate=0.02)

btt= T.tensor([[-0.6552,  0.0852,  2.0087, -0.6352,  0.4445]])
att= T.randn(1, 2)

# Nt.load_checkpoint()
btt, Nt(btt)

TypeError: ActorNetwork.__init__() got multiple values for argument 'name'